In [29]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from langchain import LLMChain, PromptTemplate
from langchain_experimental.sql import SQLDatabaseChain
import sqlalchemy
import mysql.connector

ModuleNotFoundError: No module named 'langchain_experimental'

In [6]:
# Load the model and tokenizer from Hugging Face
model_name = "t5-small"  # Or any other suitable model
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Database credentials
db_user = "root"
db_password = "1234"
db_host = "localhost"
db_name = "Auto_insurance"

# Create the SQLDatabase instance
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}", sample_rows_in_table_info=3)

# Define the prompt for MySQL
mysql_prompt = """
You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble.
"""

# Define the prompt template
prompt = PromptTemplate(template=mysql_prompt, input_variables=["question"])

# Define the function to generate SQL queries
def generate_sql_query(question):
    inputs = tokenizer(f"translate English to SQL: {question}", return_tensors="pt")
    outputs = model.generate(inputs["input_ids"], max_length=150)
    sql_query = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return sql_query

# Connect to the MySQL database
def connect_to_database():
    connection = mysql.connector.connect(
        host=db_host,
        user=db_user,
        password=db_password,
        database=db_name
    )
    return connection

# Execute the SQL query and retrieve results
def execute_query(query, connection):
    cursor = connection.cursor()
    cursor.execute(query)
    results = cursor.fetchall()
    cursor.close()
    return results

# Integrate everything into a Q&A interface
def get_answer(question):
    connection = connect_to_database()
    sql_query = generate_sql_query(question)
    results = execute_query(sql_query, connection)
    connection.close()
    return results

if __name__ == "__main__":
    question = "How many states data is available and name them?"
    answer = get_answer(question)
    print("Answer:", answer)


ImportError: Unable to convert output to PyTorch tensors format, PyTorch is not installed.